# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846821313208                   -0.70    4.0
  2   -7.852318841710       -2.26       -1.53    1.0
  3   -7.852646076844       -3.49       -2.52    3.2
  4   -7.852646676393       -6.22       -3.34    2.0
  5   -7.852646685774       -8.03       -4.76    1.0
  6   -7.852646686725       -9.02       -5.16    4.2
  7   -7.852646686729      -11.40       -5.81    1.8
  8   -7.852646686730      -12.29       -7.22    2.0
  9   -7.852646686730      -14.35       -7.55    2.5
 10   -7.852646686730   +    -Inf       -8.46    2.8
 11   -7.852646686730   +  -14.57       -9.88    2.2
 12   -7.852646686730   +    -Inf      -10.69    2.8
 13   -7.852646686730      -14.75      -11.78    2.0
 14   -7.852646686730      -14.75      -12.38    2.5


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846863951456                   -0.70           4.2
  2   -7.852529290220       -2.25       -1.62   0.80    5.0
  3   -7.852610066975       -4.09       -2.80   0.80    5.2
  4   -7.852646561247       -4.44       -3.55   0.80    7.2
  5   -7.852646683452       -6.91       -4.48   0.80    8.5
  6   -7.852646686671       -8.49       -5.08   0.80   10.5
  7   -7.852646686725      -10.27       -6.20   0.80   11.5
  8   -7.852646686730      -11.28       -7.07   0.80   13.8
  9   -7.852646686730      -13.57       -7.81   0.80   14.8
 10   -7.852646686730      -14.75       -9.18   0.80   16.0
 11   -7.852646686730   +  -15.05       -9.37   0.80   18.5
 12   -7.852646686730   +  -15.05      -10.29   0.80   18.8
 13   -7.852646686730   +  -14.57      -11.74   0.80   21.0
 14   -7.852646686730      -14.45      -12.60   0.80   23.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.407680e+01     3.504898e+00
 * time: 0.41619300842285156
     1     1.245483e+00     1.667143e+00
 * time: 0.6164929866790771
     2    -1.327056e+00     1.911917e+00
 * time: 0.6408929824829102
     3    -3.700182e+00     1.597486e+00
 * time: 0.675555944442749
     4    -5.112394e+00     1.387142e+00
 * time: 0.710439920425415
     5    -6.796190e+00     1.105682e+00
 * time: 0.7451238632202148
     6    -7.425013e+00     4.379775e-01
 * time: 0.7794928550720215
     7    -7.645885e+00     1.397564e-01
 * time: 0.8031589984893799
     8    -7.735922e+00     1.740394e-01
 * time: 0.8272190093994141
     9    -7.781633e+00     8.996309e-02
 * time: 0.8509459495544434
    10    -7.810114e+00     7.764529e-02
 * time: 0.8754458427429199
    11    -7.827636e+00     5.495873e-02
 * time: 0.9003438949584961
    12    -7.837782e+00     4.742439e-02
 * time: 0.9252169132232666
    13    -7.845909e+00     3.362812e-02
 * time: 0.94988203048

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846633471224                   -0.70    4.2
  2   -7.852298175834       -2.25       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686695                   -2.54
  2   -7.852646686730      -10.46       -5.74
  3   -7.852646686730   +  -15.05      -12.16
|ρ_newton - ρ_scf|  = 3.9001196315563156e-13
|ρ_newton - ρ_scfv| = 2.0802558779448968e-13
|ρ_newton - ρ_dm|   = 5.279146281451504e-10
